## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_Data/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Date,Humidity,Lat,Lng,Max Temp,Cloudiness,Wind Speed
0,Avarua,CK,1638654016,83,-21.2078,-159.7750,78.85,40,5.75
1,Punta Arenas,CL,1638653661,50,-53.1500,-70.9167,55.51,40,25.32
2,Harper,LR,1638654020,81,4.3750,-7.7169,79.90,6,8.05
3,Zhangjiakou,CN,1638654023,45,40.8100,114.8794,28.35,0,0.27
4,Arraial do Cabo,BR,1638654025,69,-22.9661,-42.0278,76.96,40,10.36


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City,Country,Date,Humidity,Lat,Lng,Max Temp,Cloudiness,Wind Speed
0,Avarua,CK,1638654016,83,-21.2078,-159.7750,78.85,40,5.75
2,Harper,LR,1638654020,81,4.3750,-7.7169,79.90,6,8.05
4,Arraial do Cabo,BR,1638654025,69,-22.9661,-42.0278,76.96,40,10.36
6,Rikitea,PF,1638653748,71,-23.1203,-134.9692,76.06,40,10.00
7,Vaini,TO,1638653867,78,-21.2000,-175.2000,82.56,40,13.80
...,...,...,...,...,...,...,...,...,...
544,Riberalta,BO,1638655325,69,-10.9833,-66.1000,84.92,92,0.78
545,Tual,ID,1638655328,82,-5.6667,132.7500,81.75,100,10.74
548,Jacareacanga,BR,1638655335,96,-6.2222,-57.7528,75.49,100,1.25
551,Calabozo,VE,1638655343,38,8.9242,-67.4293,88.93,51,7.05


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City          0
Country       0
Date          0
Humidity      0
Lat           0
Lng           0
Max Temp      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City,Country,Date,Humidity,Lat,Lng,Max Temp,Cloudiness,Wind Speed
0,Avarua,CK,1638654016,83,-21.2078,-159.7750,78.85,40,5.75
2,Harper,LR,1638654020,81,4.3750,-7.7169,79.90,6,8.05
4,Arraial do Cabo,BR,1638654025,69,-22.9661,-42.0278,76.96,40,10.36
6,Rikitea,PF,1638653748,71,-23.1203,-134.9692,76.06,40,10.00
7,Vaini,TO,1638653867,78,-21.2000,-175.2000,82.56,40,13.80
...,...,...,...,...,...,...,...,...,...
544,Riberalta,BO,1638655325,69,-10.9833,-66.1000,84.92,92,0.78
545,Tual,ID,1638655328,82,-5.6667,132.7500,81.75,100,10.74
548,Jacareacanga,BR,1638655335,96,-6.2222,-57.7528,75.49,100,1.25
551,Calabozo,VE,1638655343,38,8.9242,-67.4293,88.93,51,7.05


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp" , "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Avarua,CK,78.85,-21.2078,-159.7750,
2,Harper,LR,79.90,4.3750,-7.7169,
4,Arraial do Cabo,BR,76.96,-22.9661,-42.0278,
6,Rikitea,PF,76.06,-23.1203,-134.9692,
7,Vaini,TO,82.56,-21.2000,-175.2000,
9,Puerto Ayora,EC,75.16,-0.7393,-90.3518,
10,Bandarbeyla,SO,75.63,9.4942,50.8122,
13,São João da Barra,BR,77.09,-21.6403,-41.0511,
14,Kavaratti,IN,80.58,10.5669,72.6420,
20,Hithadhoo,MV,81.23,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.") 
        

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Avarua,CK,78.85,-21.2078,-159.7750,Paradise Inn
2,Harper,LR,79.90,4.3750,-7.7169,Screensaver
4,Arraial do Cabo,BR,76.96,-22.9661,-42.0278,Pousada Porto Praia
6,Rikitea,PF,76.06,-23.1203,-134.9692,People ThankYou
7,Vaini,TO,82.56,-21.2000,-175.2000,Keleti Beach Resort
9,Puerto Ayora,EC,75.16,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Bandarbeyla,SO,75.63,9.4942,50.8122,JABIR HOTEL
13,São João da Barra,BR,77.09,-21.6403,-41.0511,Pousada Porto De Canoas
14,Kavaratti,IN,80.58,10.5669,72.6420,Hotels in Lakshadweep Islands
20,Hithadhoo,MV,81.23,-0.6000,73.0833,Scoop Guest House


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd></dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))